# Imports and Constants

In [1]:
import os
import random
import numpy as np
import pandas as pd
from sklearn import model_selection
from matplotlib import pyplot as plt

import PIL
# https://github.com/tensorflow/tensorflow/issues/46840
# https://stackoverflow.com/a/68239032/12405558
if PIL.__version__ == "8.3.0":
    raise ImportError(f"PIL 8.3.0 is incompatible with tensorflow 2.6.0")

BASE_DATA_DIR = "/data"
TRAIN_DIR = os.path.join(BASE_DATA_DIR, "train")
TEST_DIR = os.path.join(BASE_DATA_DIR, "test")

IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNEL = 3
IMG_SIZE = (IMG_WIDTH, IMG_HEIGHT)
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)

# Split training into training and validation dataset
VALIDATION_SIZE = 0.2

BATCH_SIZE = 20
EPOCHS = 10

TYPE = {"cat": 0, "dog": 1}


# Setup for Google Colab

In [2]:
try:
    from google.colab import drive
    from zipfile import ZipFile

    abspath_join = lambda a, *p: os.path.abspath(os.path.join(a, *p))
    MOUNT = "/content/drive"
    drive.mount(MOUNT, force_remount=True)

    BASE_DATA_DIR = abspath_join(MOUNT, "MyDrive/AI/data")
    TRAIN_DIR = abspath_join(BASE_DATA_DIR, "train")
    TEST_DIR = abspath_join(BASE_DATA_DIR, "test1")
    for zip_name in ("test1.zip","train.zip"):
        try:
            zip_dir = abspath_join(BASE_DATA_DIR, zip_name)
            # with ZipFile(zip_dir, "r") as zip_ref:
            #     print(f"Unzipping {zip_name}")
            #     zip_ref.extractall(BASE_DATA_DIR)
            # os.remove(zip_dir)
        except:
            print(f"{zip_name} not found")
except ImportError:
    pass

Mounted at /content/drive
Unzipping test1.zip
Unzipping train.zip


# Prepare Training Data

In [12]:
train_set = os.listdir(TRAIN_DIR)
types = [filename.split(".")[0] for filename in train_set]
df = pd.DataFrame({"filename": train_set, "class": types})

train_df, val_df = model_selection.train_test_split(df, test_size=VALIDATION_SIZE)

TOTAL_TRAIN = train_df.shape[0]
TOTAL_VAL = val_df.shape[0]

12500

## Some samples

In [10]:
df["class"].value_counts() #.rename({v: k for k, v in TYPE.items()})


cat    12500
dog    12500
Name: class, dtype: int64

In [5]:
print(f"Training: {len(train_df)}")
train_df.reset_index(drop=True)["class"].value_counts()


Training: 20000


dog    10057
cat     9943
Name: class, dtype: int64

In [6]:
print(f"Validation: {len(val_df)}")
val_df.reset_index(drop=True)["class"].value_counts()

Validation: 5000


cat    2557
dog    2443
Name: class, dtype: int64

# Build Model

In [13]:
from tensorflow.keras import models, layers, losses

# https://www.tensorflow.org/guide/keras/functional
img_input = layers.Input(shape=IMG_SHAPE)

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, (3, 3), activation="relu")(img_input)
x = layers.MaxPool2D((2, 2))(x)  # Stack layer

# Same as previous convolution but with 32 filters
x = layers.Conv2D(32, (3, 3), activation="relu")(x)
x = layers.MaxPooling2D((2, 2))(x)

# Same as previous convolution but with 64 filters
x = layers.Conv2D(64, (3, 3), activation="relu")(x)
x = layers.MaxPooling2D((2, 2))(x)

# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation="relu")(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation="sigmoid")(x)

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully
# connected layer + sigmoid output layer
model = models.Model(img_input, output)

model.compile(loss="binary_crossentropy", metrics=["accuracy"])

model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 126, 126, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0     

# Data Preprocessing

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

GEN_PARAMS = {
    "directory": TRAIN_DIR,
    "target_size": IMG_SIZE,
    "batch_size": BATCH_SIZE,
    "class_mode": "binary",
}

train_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_gen = train_datagen.flow_from_dataframe(train_df, **GEN_PARAMS)
val_gen = val_datagen.flow_from_dataframe(val_df, **GEN_PARAMS)


Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


# Training

In [ ]:
history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    validation_steps=TOTAL_VAL // BATCH_SIZE,
    steps_per_epoch=TOTAL_TRAIN // BATCH_SIZE,
)


Epoch 1/10
 295/1000 [=======>......................] - ETA: 4:38 - loss: 0.6664 - accuracy: 0.6220